# Timer

In [1]:
from time import sleep
from datetime import datetime as dt

start_time = "14.09.2023, 9:30"
start_time = dt.strptime(start_time, "%d.%m.%Y, %H:%M")

while True:
    if start_time < dt.now():
        break
    else:
        sleep(10)

# Parameter optimization

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

ttype = 'sell'
pattern = ['STOCH', 'RSI']#, 'Trend']
indicator_list = pattern
indicator_list_higher = []#['Trend']

work_timeframe = '1h'
higher_timeframe = '4h'
opt_limit = 10000
load = False

print(f'Timeframe is {work_timeframe}/{higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Indicator_list'] = indicator_list
configs['Higher_TF_indicator_list'] = indicator_list_higher
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'RSI': {
                        'timeperiod': [14], 
                        'low_bound': [15, 20, 25, 30, 35]
                       },
                'STOCH': {
                          'fastk_period': [9],
                          'slowk_period': [7],
                          'slowd_period': [3], 
                          'low_bound': [10, 15, 20, 25]
                        },
                'Trend': {
                          'timeperiod': [30], 
                          'low_bound': [0]
                        }
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load, 'optimize')

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Timeframe is 1h/4h, trade type is sell
Number of combinations is 20


100%|██████████| 20/20 [26:08<00:00, 78.41s/it] 


# Check local statistics

In [4]:
# rsi + stoch
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg')
stat.head(20)

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
17,STOCH_RSI,14,35,9,7,3,15,2471,0.850483,3.027083,-369.455683,7479.922917,0.9823,0.9232,0.9044,0.9162,0.9190,0.8990,0.8993,0.9071,0.8950,0.8624,0.8132,0.7807,0.7687,0.7759,0.7874,0.8010,0.8024,0.8105,0.8253,0.8318,0.8298,0.8282,0.8234,0.8254,0.30,0.59,0.89,1.17,1.46,1.74,2.02,2.30,2.58,2.86,3.14,3.41,3.65,3.84,3.99,4.11,4.21,4.28,4.33,4.36,4.36,4.36,4.35,4.35
19,STOCH_RSI,14,35,9,7,3,25,5390,0.826400,3.044583,-935.704000,16410.304167,0.9744,0.9265,0.8991,0.9109,0.9088,0.8950,0.8838,0.8814,0.8674,0.8351,0.8037,0.7726,0.7607,0.7642,0.7687,0.7728,0.7730,0.7774,0.7802,0.7774,0.7770,0.7773,0.7727,0.7735,0.31,0.62,0.92,1.20,1.49,1.78,2.06,2.34,2.61,2.89,3.16,3.43,3.66,3.86,4.01,4.13,4.22,4.29,4.34,4.36,4.36,4.35,4.34,4.34
18,STOCH_RSI,14,35,9,7,3,20,4239,0.824417,3.056250,-744.297750,12955.443750,0.9711,0.9125,0.8858,0.8981,0.8985,0.8896,0.8800,0.8806,0.8683,0.8350,0.7975,0.7631,0.7543,0.7606,0.7672,0.7736,0.7733,0.7778,0.7860,0.7848,0.7842,0.7849,0.7791,0.7801,0.31,0.61,0.91,1.20,1.48,1.77,2.05,2.33,2.61,2.89,3.17,3.44,3.67,3.87,4.02,4.15,4.24,4.32,4.36,4.39,4.40,4.39,4.38,4.39
16,STOCH_RSI,14,35,9,7,3,10,642,0.794308,3.226667,-132.054050,2071.520000,0.9348,0.8933,0.8864,0.8970,0.8960,0.8476,0.8569,0.8814,0.8514,0.8174,0.7539,0.7063,0.6826,0.6917,0.7066,0.7203,0.7219,0.7389,0.7579,0.7713,0.7657,0.7612,0.7593,0.7636,0.30,0.61,0.91,1.21,1.51,1.81,2.11,2.40,2.70,3.00,3.31,3.61,3.86,4.06,4.24,4.38,4.50,4.59,4.66,4.70,4.72,4.74,4.75,4.76
12,STOCH_RSI,14,30,9,7,3,10,481,0.761325,3.674167,-114.802675,1767.274167,0.8472,0.8198,0.8361,0.8798,0.8879,0.8488,0.8634,0.8783,0.8517,0.8120,0.7413,0.6914,0.6595,0.6547,0.6631,0.6829,0.6836,0.6937,0.7058,0.7229,0.7169,0.7114,0.7103,0.7093,0.35,0.71,1.06,1.40,1.74,2.07,2.40,2.73,3.06,3.40,3.75,4.08,4.36,4.60,4.80,4.97,5.11,5.22,5.31,5.36,5.40,5.42,5.43,5.45
13,STOCH_RSI,14,30,9,7,3,15,1536,0.781679,3.723333,-335.340800,5719.040000,0.8441,0.7836,0.7909,0.8350,0.8554,0.8438,0.8558,0.8650,0.8494,0.8146,0.7591,0.7250,0.7064,0.7035,0.7142,0.7369,0.7373,0.7450,0.7618,0.7737,0.7694,0.7665,0.7615,0.7624,0.37,0.74,1.10,1.46,1.81,2.15,2.49,2.82,3.15,3.49,3.83,4.15,4.43,4.67,4.87,5.03,5.16,5.26,5.33,5.38,5.40,5.42,5.42,5.43
14,STOCH_RSI,14,30,9,7,3,20,2363,0.766900,3.864583,-550.815300,9132.010417,0.8548,0.7977,0.7838,0.8201,0.8374,0.8361,0.8416,0.8438,0.8346,0.7996,0.7547,0.7154,0.7004,0.6994,0.7060,0.7202,0.7226,0.7247,0.7346,0.7397,0.7373,0.7371,0.7310,0.7330,0.40,0.79,1.17,1.53,1.90,2.25,2.60,2.95,3.29,3.64,3.98,4.30,4.60,4.85,5.04,5.21,5.34,5.45,5.52,5.57,5.59,5.60,5.59,5.59
15,STOCH_RSI,14,30,9,7,3,25,2817,0.760487,3.893333,-674.706713,10967.520000,0.8584,0.7992,0.7916,0.8308,0.8438,0.8336,0.8358,0.8387,0.8282,0.7908,0.7508,0.7135,0.6986,0.6951,0.6983,0.7108,0.7128,0.7156,0.7180,0.7209,0.7189,0.7181,0.7138,0.7156,0.40,0.80,1.19,1.55,1.92,2.28,2.63,2.98,3.32,3.67,4.01,4.33,4.63,4.88,5.08,5.24,5.38,5.48,5.56,5.61,5.63,5.63,5.62,5.62
8,STOCH_RSI,14,25,9,7,3,10,319,0.791117,4.123333,-66.633783,1315.343333,0.8322,0.7899,0.8457,0.9136,0.9436,0.9182,0.9561,0.9648,0.9451,0.8946,0.7994,0.7327,0.6838,0.6725,0.6752,0.6941,0.6978,0.7053,0.7187,0.7317,0.7245,0.7193,0.7137,0.7143,0.41,0.82,

In [2]:
# rsi + stoch + trend
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg')
stat.head(20)

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
0,STOCH_RSI_Trend,14,15,9,7,3,10,30,0,0,0.000000,0.000000,-0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
19,STOCH_RSI_Trend,14,35,9,7,3,25,30,0,1078,1.137329,2.042917,148.040842,2202.264167,1.5436,1.5383,1.5246,1.5152,1.4527,1.4234,1.3571,1.3304,1.2491,1.1529,1.0733,0.9994,0.9862,0.9865,0.9771,0.9546,0.9442,0.9344,0.9179,0.9061,0.8978,0.8862,0.8751,0.8698,0.20,0.39,0.57,0.75,0.92,1.10,1.28,1.46,1.66,1.86,2.07,2.27,2.45,2.59,2.71,2.81,2.89,2.94,2.98,3.00,3.01,3.02,3.04,3.06
18,STOCH_RSI_Trend,14,35,9,7,3,20,30,0,903,1.088146,2.142083,79.595688,1934.301250,1.4508,1.4154,1.4152,1.4228,1.3817,1.3645,1.3191,1.2940,1.2144,1.1167,1.0299,0.9567,0.9526,0.9560,0.9433,0.9207,0.9129,0.9035,0.8843,0.8703,0.8608,0.8494,0.8427,0.8378,0.20,0.40,0.59,0.77,0.95,1.13,1.32,1.51,1.72,1.93,2.15,2.37,2.55,2.71,2.84,2.94,3.03,3.09,3.14,3.17,3.19,3.21,3.23,3.27
17,STOCH_RSI_Trend,14,35,9,7,3,15,30,0,573,1.061271,2.283750,35.108188,1308.588750,1.3621,1.3948,1.3911,1.3984,1.3904,1.3898,1.3241,1.2939,1.1979,1.0753,0.9756,0.9269,0.9286,0.9352,0.9155,0.8875,0.8813,0.8671,0.8545,0.8414,0.8245,0.8091,0.8043,0.8012,0.21,0.42,0.62,0.81,1.00,1.19,1.39,1.59,1.81,2.05,2.29,2.52,2.71,2.88,3.02,3.14,3.23,3.31,3.36,3.39,3.42,3.45,3.48,3.52
16,STOCH_RSI_Trend,14,35,9,7,3,10,30,0,144,0.978142,2.694167,-3.147600,387.960000,1.0698,1.1377,1.2033,1.3722,1.3677,1.3690,1.2809,1.2155,1.0850,0.9821,0.9031,0.8464,0.8549,0.8560,0.8558,0.8297,0.8244,0.8145,0.8137,0.8038,0.7738,0.7475,0.7392,0.7294,0.23,0.46,0.69,0.91,1.13,1.34,1.57,1.81,2.07,2.35,2.64,2.90,3.13,3.34,3.52,3.68,3.81,3.93,4.02,4.09,4.16,4.22,4.29,4.37
15,STOCH_RSI_Trend,14,30,9,7,3,25,30,0,390,1.087504,2.987083,34.126625,1164.962500,1.0662,1.2480,1.3934,1.5442,1.6000,1.5907,1.5606,1.5082,1.3570,1.1658,1.0540,0.9601,0.9306,0.9098,0.8809,0.8581,0.8519,0.8416,0.8258,0.8187,0.8012,0.7854,0.7755,0.7724,0.29,0.57,0.83,1.08,1.32,1.56,1.80,2.04,2.32,2.61,2.91,3.20,3.46,3.69,3.89,4.07,4.22,4.34,4.43,4.49,4.56,4.61,4.67,4.73
14,STOCH_RSI_Trend,14,30,9,7,3,20,30,0,354,1.038692,3.022500,13.696850,1069.965000,1.0072,1.1534,1.2880,1.4625,1.5405,1.5417,1.5207,1.4771,1.3211,1.1279,1.0119,0.9171,0.8905,0.8715,0.8403,0.8203,0.8150,0.8046,0.7858,0.7742,0.7542,0.7403,0.7327,0.7301,0.29,0.57,0.83,1.08,1.32,1.56,1.80,2.05,2.32,2.63,2.93,3.23,3.49,3.73,3.93,4.11,4.27,4.39,4.49,4.57,4.64,4.70,4.77,4.84
13,STOCH_RSI_Trend,14,30,9,7,3,15,30,0,280,1.016438,3.132083,4.602500,876.983333,0.9762,1.0983,1.2240,1.4512,1.5444,1.5602,1.5287,1.4800,1.2987,1.0938,0.9565,0.9000,0.8782,0.8578,0.8115,0.7909,0.7843,0.7741,0.7674,0.7525,0.7297,0.7159,0.7108,0.7094,0.29,0.58,0.85,1.10,1.35,1.59,1.84,2.09,2.38,2.70,3.02,3.33,3.61,3.86,4.08,4.27,4.43,4.57,4.67,4.76,4.83,4.91,4.99,5.07
12,STOCH_RSI_Trend,14,30,9,7,3,10,30,0,98,0.954350,3.194583,-4.473700,313.069167,0.9780,1.0146,1.1231,1.4365,1.4465,1.4751,1.3873,1.2828,1.1454,1.0085,0.9003,0.8

In [2]:
# trend
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg')
stat.head(20)

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
0,STOCH_RSI_Trend,14,15,9,7,3,10,48,0,5,1.451137,-605.122083,2.255687,-3025.610417,0.8271,0.3377,0.5235,1.9639,1.7967,2.7524,2.5468,1.4078,1.4898,1.4894,1.4588,1.4268,1.4081,1.4011,1.4011,1.4011,1.4011,1.4011,1.4011,1.2964,1.2964,1.2718,1.5686,1.5587,-1.61,-3.44,-5.54,-8.04,-11.05,-14.78,-19.49,-25.59,-34.12,-46.64,-66.98,-105.85,-208.17,-1270.68,-1278.33,-1280.99,-1278.76,-1277.35,-1272.89,-1269.13,-1266.45,-1262.20,-1258.56,-1256.29
1,STOCH_RSI_Trend,14,15,9,7,3,15,48,0,7,1.746588,-436.352500,5.226113,-3054.467500,1.0613,0.8431,1.2474,2.5965,2.3442,3.1245,2.9246,1.7007,1.7498,1.7239,1.6908,1.6561,1.6261,1.6186,1.6186,1.6186,1.6186,1.6186,1.6186,1.5054,1.5054,1.4786,1.7354,1.6927,-1.39,-2.99,-4.81,-6.91,-9.38,-12.38,-16.09,-20.79,-27.25,-36.57,-51.49,-79.66,-153.16,-912.52,-918.34,-920.60,-919.35,-918.59,-915.67,-913.25,-911.61,-908.78,-906.27,-904.61
2,STOCH_RSI_Trend,14,15,9,7,3,20,48,0,9,1.596113,-340.005417,5.365013,-3060.048750,1.2108,0.9117,1.3064,2.4993,2.2864,2.4675,2.3047,1.5210,1.5073,1.5251,1.5576,1.5308,1.5076,1.5018,1.5018,1.4998,1.4840,1.4840,1.4724,1.3866,1.3866,1.3594,1.5711,1.5230,-1.15,-2.44,-3.92,-5.61,-7.59,-9.97,-12.90,-16.61,-21.67,-28.96,-40.63,-62.59,-119.81,-710.47,-715.04,-716.84,-715.90,-715.35,-713.12,-711.26,-710.01,-707.83,-705.88,-704.58
3,STOCH_RSI_Trend,14,15,9,7,3,25,48,0,10,1.594075,-305.912083,5.940750,-3059.120833,1.2035,0.9091,1.3025,2.4921,2.2808,2.4627,2.3013,1.5199,1.5063,1.5242,1.5566,1.5299,1.5067,1.5009,1.5009,1.4989,1.4833,1.4833,1.4717,1.3859,1.3859,1.3588,1.5703,1.5223,-1.00,-2.13,-3.44,-4.93,-6.69,-8.81,-11.44,-14.78,-19.34,-25.93,-36.47,-56.27,-107.78,-639.38,-643.48,-645.09,-644.25,-643.74,-641.74,-640.08,-638.94,-636.96,-635.19,-634.03
4,STOCH_RSI_Trend,14,20,9,7,3,10,48,0,20,1.249292,-153.590000,4.985833,-3071.800000,0.6617,0.3524,0.7480,1.3337,1.3162,1.6128,1.5756,1.2534,1.2242,1.2382,1.2101,1.2208,1.2792,1.2815,1.2967,1.2815,1.2991,1.3075,1.3627,1.3172,1.2724,1.2904,1.5646,1.6831,-0.63,-1.30,-2.04,-2.89,-3.86,-5.01,-6.40,-8.15,-10.49,-13.84,-19.13,-29.05,-54.82,-320.63,-322.63,-323.37,-322.88,-322.58,-321.51,-320.60,-319.95,-318.90,-318.01,-317.49
5,STOCH_RSI_Trend,14,20,9,7,3,15,48,0,42,1.390696,-75.720417,16.409225,-3180.257500,0.9365,1.0822,1.4411,1.8587,1.7521,1.8785,1.7716,1.4902,1.4213,1.3947,1.3679,1.3497,1.3660,1.3588,1.3193,1.2800,1.2728,1.2621,1.2733,1.2248,1.1959,1.2360,1.3913,1.4519,-0.51,-1.06,-1.66,-2.30,-3.00,-3.78,-4.68,-5.75,-7.10,-8.92,-11.66,-16.61,-29.09,-155.86,-156.95,-157.44,-157.33,-157.30,-156.90,-156.55,-156.31,-155.87,-155.46,-155.20
6,STOCH_RSI_Trend,14,20,9,7,3,20,48,0,54,1.404287,-59.775000,21.831525,-3227.850000,1.0010,1.0718,1.4281,1.7412,1.6725,1.7458,1.7022,1.4960,1.4124,1.4082,1.4393,1.4288,1.4569,1.4500,1.3863,1.3547,1.3558,1.2896,1.2954,1.2553,1.2311,1.2618,1.3869,1.4318,-0.48,-0.99,-1.54,-2.11,-2.74,-3.43,-4.21,-5.11,-6.24,-7.73,-9.95,-13.86,-23.64,-122.30,-123.20,-123.63,-123.59,-123.60,-123.33,-123.09,-122.92,-122.58,-122.27,-122.06
8,STOCH_RSI_Trend,14,25,9,7,3,10,48,0,56,0.760625,-56.328333,-13.405000,-3154.386667,0.9792,0.7504,0.7772,0.8080,0.7601,0.8178,0.7194,0.6687,0

# Research parameter influence

In [81]:
param = 'Trend_timeperiod'

res = stat.groupby([param]).agg({'e_ratio_avg': 'mean',
                                 'pct_price_diff_avg': 'mean',
                                 'e_ratio_rank': 'mean', 
                                 'price_rank': 'mean', 
                                 'forecasts_num': 'sum'}).sort_values(param).reset_index()
res

,Trend_timeperiod,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,forecasts_num
0,2,1.937883,-0.562604,65.569898,-39.616042,247
1,3,2.747457,-1.018333,76.114432,-44.385000,123
2,4,4.568197,-2.099375,56.411997,-33.292188,91
3,5,3.548132,-1.371806,74.680826,-39.263194,80
4,6,3.757032,-1.289444,80.305944,-37.298889,80
5,8,3.517467,-1.098056,77.010065,-33.492500,86


# Plot result in file

In [77]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

x, y = 'Trend_timeperiod', 'pct_price_diff_avg'
ax.plot(res[x], res[y])
ax.set_xlabel(x)
ax.set_ylabel(y)
ax.set_title('_'.join(pattern) + '_' + ttype)

fig.savefig('temp.png', dpi=fig.dpi)

# Save new config data to config file

In [3]:
from config_updater import ConfigUpdater

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                'RSI': {
                        'timeperiod': [14], 
                        'low_bound': [20]
                       },
                'STOCH': {
                          'fastk_period': [9],
                          'slowk_period': [7],
                          'slowd_period': [3], 
                          'low_bound': [25]
                        },
                'Trend': {
                          'timeperiod': [48], 
                          'low_bound': [0]
                        }
             }
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)

# Check global statistics

In [3]:
import numpy as np
import pandas as pd
from glob import glob

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '5m'
higher_timeframe = '1h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['RSI_timeperiod', 
                           'RSI_low_bound', 
                           'STOCH_fastk_period', 
                           'STOCH_slowk_period', 
                           'STOCH_slowd_period', 
                           'STOCH_low_bound']).agg({'pct_right_forecast_avg': 'mean',
                                                    'pct_price_diff_avg': 'mean',
                                                    'forecast_rank': 'mean', 
                                                    'price_rank': 'mean', 
                                                    'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                         ascending=False)
total_stat.head(20)

pct_right_forecast_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                           
16             30            9                  7                  3                  30                            83.541667   
                                                                                      25                            85.785833   
12             25            9                  7                  3                  30                            88.782083   
14             30            9                  7                  3                  25                            80.833333   
12             25            9                  7                  3                  25                            87.847917   
14             25            9                  7                  3                  30                            89.395000   
                                                                                      20                            89.395000   
                                                                                      25                            89.395000   
12             25            9                  7                  3                  20                            89.395000   
16             30            9                  7                  3                  20                            84.820833   
14             30            9                  7                  3                  30                            78.804583   
16             25            9                  7                  3                  20                            88.750000   
                                                                                      25                            88.750000   
                                                                                      30                            88.750000   
12             30            9                  7                  3                  30                            75.416667   
14             30            9                  7                  3                  20                            79.687500   
16             15            5                  4                  3                  25                            93.229167   
12             25            9                  7                  3                  15                            91.664583   
14             25            9                  7                  3                  15                            91.664583   
16             15            7                  4                  5                  25                            95.025694   

                                                                                                       pct_price_diff_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                       
16             30            9                  7                  3                  30                        -1.760000   
                                                                                      25                        -1.970833   
12             25            9                  7                  3                  30                        -2.084167   
14             30            9                  7                  3                  25                        -1.777917   
12             25            9                  7                  3                  25                        -2.421250   
14             25            9                  7                  3                  30                        -2.759583   
                                                                                      20                        -2.759583   
                                                                                      25                        -2.759583   
12            